<a href="https://colab.research.google.com/github/philpham8/FindRepresentativeRNAChains/blob/main/FindRepresentativeRNAChains_Phillip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import requests
import pandas as pd

def json_from_url(url):
  response = requests.get(url, timeout=5)

  if response.status_code == requests.codes.ok: return response.json()
  else: return

# Fetch pubmed on requested PDB entry
def get_pdb_pubmed(pdb_id):
  request_url = "https://data.rcsb.org/rest/v1/core/pubmed"
  return json_from_url(request_url + '/' + pdb_id)

# Fetch information on requested PDB entry
def get_pdb_entry(pdb_id):
  request_url = "https://data.rcsb.org/rest/v1/core/entry"
  return json_from_url(request_url + '/' + pdb_id)

# Fetch information on requested PDB entity/chain (e.g. individual RNA)
def get_pdb_entity(pdb_id, chain_id):
  request_url = "https://data.rcsb.org/rest/v1/core/polymer_entity"
  return json_from_url(request_url + '/' + pdb_id + '/' + chain_id)

# Fetch abstract title from pubmed
def pdb_id_contains_excluded_words(pdb_id, excluded_words):
    pubmed_json = get_pdb_pubmed(pdb_id)
    if pubmed_json is not None and 'rcsb_pubmed_abstract_text' in pubmed_json:
      abstract = pubmed_json['rcsb_pubmed_abstract_text'].lower()
      return any(excluded_word.lower() in abstract for excluded_word in excluded_words)
    else: return False

# Checks that the polymer is of type RNA and DOES NOT contain DNA, oligosaccharide, or protein.
def contains_rna_only(entry):
    return entry['rcsb_entry_info']['polymer_composition'] == 'RNA' and entry['rcsb_entry_info']['na_polymer_entity_types'] == 'RNA (only)'

# Checks if it contains any nonpolymer (either ligands or ions)
# TODO: Ions should be permitted while ligands should not be. Temp action is excluding both.
def contains_nonpolymer(entry):
    return entry['rcsb_entry_info']['nonpolymer_entity_count'] > 0

# Retrieve list of chain ids from entry.
def get_list_of_chain_id(entry):
    return entry['rcsb_entry_container_identifiers']['polymer_entity_ids']

# Determine polymer length given entity.
def get_polymer_length(entity):
    return entity['entity_poly']['rcsb_sample_sequence_length']


**Run following cell and input direct URL to CSV found on BGSU RNA**

In [101]:
# Ask URL to CSV and then create dataframe from .csv
# E.g. http://rna.bgsu.edu/rna3dhub/nrlist/download/3.180/3.0A/csv
csv_url = input('Please provide direct URL to desired CSV file found on rna.bgsu.edu: ')
data = pd.read_csv(csv_url, usecols=[1])

# Ask user for maximum nucleotides
nt_max_cutoff = int(input("What should the maximium number (non-inclusive) of ribonucleotides be? "))

# Ask user for minimum nucleotides
nt_min_cutoff = int(input("What should the minimum number (non-inclusive) of ribonucleotides be? "))

# Ask user for minimum nucleotides
excluded_words = input("List any words that you want to blacklist in the PubMed abstract. Separate words by space: ").split()

Please provide direct URL to desired CSV file found on rna.bgsu.edu: /content/nrlist_3.179_3.0A.csv
What should the maximium number (non-inclusive) of ribonucleotides be? 400
What should the minimum number (non-inclusive) of ribonucleotides be? 100
List any words that you want to blacklist in the PubMed abstract. Separate words by space: tRNA


**Filter all RNAs based on max/min nt cutoffs. Includes nonpolymers (ions/ligands). Excludes carbohydrates and 
proteins.**

In [102]:
# Running list of chains that meet criteria (no proteins, no carbohydrates, no ligand/ion/water, and less than nt_cutoff)
matching_chains = []

# Grab second column (the 'representative' column) from .csv and grab the PDB id.
# TODO: Use regex instead
for i, row in data.iterrows():
  pdb_id = row[0].split('|')[0] 

  # Immediately skip entry if it contains the word tRNA or other excluded words
  entry = get_pdb_entry(pdb_id) 

  ## Checks if RNA and exclude any nonpolymers. If true, fetch all chains and check if it is less than nt_cutoff.
  ## if contains_rna_only(entry) and not contains_nonpolymer(entry):
  
  # Checks if RNA. Also includes all nonpolymers.
  if contains_rna_only(entry):
    if pdb_id_contains_excluded_words(pdb_id, excluded_words): 
      print('Skipping PDA ID because it contains exclusion word: ', pdb_id)
      continue
    list_of_chain_id = get_list_of_chain_id(entry)

    for chain_id in list_of_chain_id:
      entity = get_pdb_entity(pdb_id, chain_id)
      if nt_min_cutoff < get_polymer_length(entity) < nt_max_cutoff:
        matching_chains.append(pdb_id + '|' + chain_id)

# Print number of RNAs found
print()
print('Found ', len(matching_chains), ' molecules:')
print(matching_chains)

Skipping PDA ID because it contains exclusion word:  5L4O
Skipping PDA ID because it contains exclusion word:  4P5J
Skipping PDA ID because it contains exclusion word:  4JRC
Skipping PDA ID because it contains exclusion word:  3RG5
Skipping PDA ID because it contains exclusion word:  5HNJ
Skipping PDA ID because it contains exclusion word:  3BNR
Skipping PDA ID because it contains exclusion word:  3BNT
Skipping PDA ID because it contains exclusion word:  6UGI
Skipping PDA ID because it contains exclusion word:  4U35
Skipping PDA ID because it contains exclusion word:  2V6W
Skipping PDA ID because it contains exclusion word:  1YFG
Skipping PDA ID because it contains exclusion word:  4RZD
Skipping PDA ID because it contains exclusion word:  6CU1
Skipping PDA ID because it contains exclusion word:  4JF2
Skipping PDA ID because it contains exclusion word:  6PMO
Skipping PDA ID because it contains exclusion word:  2V7R
Skipping PDA ID because it contains exclusion word:  435D
Skipping PDA I

**Helper function to determine list of ligands present in BGSU dataset:**

In [ ]:
# If entry contains nonpolymer AND a list of bound components is given, display it.
# Notice: Some entries have nonpolymer entities but fail to include a list.
def add_nonpolymers(entry):
    if contains_nonpolymer(entry) and 'nonpolymer_bound_components' in entry['rcsb_entry_info']:
        nonpolymers = entry['rcsb_entry_info']['nonpolymer_bound_components']

        for nonpolymer in nonpolymers:
          if nonpolymer not in list_of_nonpolymers:
            list_of_nonpolymers.append(nonpolymer)

list_of_nonpolymers = []

# Grab second column (the 'representative' column) from .csv and grab the PDB id.
# TODO: Use regex instead
for i, row in data.iterrows():
  pdb_id = row[0].split('|')[0] 
  entry = get_pdb_entry(pdb_id) 

  add_nonpolymers(entry)

# Print number of RNAs found
print()
print('Found ', len(list_of_nonpolymers), ' nonpolymers')
print(list_of_nonpolymers)


Found  115  nonpolymers
['ATP', 'K', 'MG', 'ZN', 'SF4', '5GP', 'FME', 'GNP', 'GTP', 'FE2', 'OH', 'TAC', 'MN', 'GCP', 'ARG', 'NA', 'CA', 'HIS', 'M5Z', 'NAD', 'F86', 'POP', 'PLR', 'SAH', 'ANP', 'IDQ', 'NAG', 'SO4', 'LYS', 'DUT', 'UDP', 'ADP', 'OHX', 'CD', 'MLI', 'SR', 'MET', 'U6A', 'BA', 'ALF', 'NI', 'GDP', 'AMZ', 'CAC', 'CS', '2KH', 'BEF', 'PYI', 'FMT', 'CH1', 'YQM', 'PAR', 'SPE', 'B12', 'MMC', 'A', 'RQA', 'AU', 'DCT', 'CYS', 'APC', 'CDP', 'MUB', 'IUM', 'ACT', '0G6', 'CCC', 'G5J', 'DCP', 'FES', 'BTN', 'AG', 'M4H', 'IPA', 'FOU', 'HG', 'PPV', 'TPP', 'PO4', 'DTP', 'ACP', 'BR', 'NCO', 'CO', 'IR', 'YMP', 'M7G', 'LI', 'T1C', 'PRP', 'A23', 'MGF', 'GH3', 'GLN', 'CSG', 'G7M', '2AD', 'NVA', 'NPM', 'GE6', 'UTP', 'G88', 'S9L', 'TSB', 'AMP', 'GZG', 'CTC', 'GNG', 'MGT', 'G4P', 'TL', 'U3H', '4PQ', 'MES', 'SAM']
